# Обучение N-gram языковой модели с BPE токенизатором


In [2]:
import re
import json
import random
import math
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Optional

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import regex

# Для воспроизводимости
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

## 1. BPE Токенизатор

Используем токенизатор из предыдущего задания с паттерном претокенизации GPT-4 (cl100k_base).

In [ ]:
class BPETokenizer:    
    PRETOKENIZE_PATTERN = regex.compile(
        r"'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}|\ ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+",
        regex.UNICODE
    )
    
    def __init__(self, vocab_size: int = 10000, special_tokens: List[str] = None):
        self.vocab_size = vocab_size
        
        if special_tokens is None:
            special_tokens = ["<PAD>", "<UNK>", "<BOS>", "<EOS>", "<MASK>"]
        self.special_tokens = special_tokens
        
        self.vocab: Dict[str, int] = {}
        self.inverse_vocab: Dict[int, str] = {}
        self.merges: List[Tuple[str, str]] = []
        self.merge_ranks: Dict[Tuple[str, str], int] = {}  # Для быстрого поиска
        self.cache: Dict[str, List[int]] = {}  # Кэш токенизации
        
        self._init_base_vocab()
        
    def _init_base_vocab(self):
        for i, token in enumerate(self.special_tokens):
            self.vocab[token] = i
            self.inverse_vocab[i] = token
        
        offset = len(self.special_tokens)
        for i in range(256):
            byte_token = bytes([i]).decode('latin-1')
            self.vocab[byte_token] = offset + i
            self.inverse_vocab[offset + i] = byte_token
            
    def _pretokenize(self, text: str) -> List[str]:
        tokens = self.PRETOKENIZE_PATTERN.findall(text)
        return tokens if tokens else list(text)
    
    def _text_to_bytes(self, text: str) -> Tuple[str, ...]:
        return tuple(bytes([b]).decode('latin-1') for b in text.encode('utf-8'))
    
    def _build_merge_ranks(self):
        self.merge_ranks = {merge: i for i, merge in enumerate(self.merges)}
    
    def train(self, texts: List[str], verbose: bool = True):
        if verbose:
            print("Претокенизация и подсчет частот...")
        
        word_freqs = Counter()
        for text in tqdm(texts, disable=not verbose):
            pretokens = self._pretokenize(text)
            for pretoken in pretokens:
                byte_seq = self._text_to_bytes(pretoken)
                word_freqs[byte_seq] += 1
        
        if verbose:
            print(f"Уникальных претокенов: {len(word_freqs)}")
            print("Построение индекса пар...")
        
        pair_freqs = Counter()
        pair_to_words = defaultdict(set)
        word_to_pairs = {}
        
        for word in word_freqs:
            if len(word) < 2:
                continue
            pairs_in_word = []
            for i in range(len(word) - 1):
                pair = (word[i], word[i + 1])
                pair_freqs[pair] += word_freqs[word]
                pair_to_words[pair].add(word)
                pairs_in_word.append((i, pair))
            word_to_pairs[word] = pairs_in_word
        
        num_merges = self.vocab_size - len(self.vocab)
        
        if verbose:
            print(f"Выполняем {num_merges} слияний...")
        
        pbar = tqdm(range(num_merges), disable=not verbose)
        
        for _ in pbar:
            if not pair_freqs:
                break
            
            best_pair = pair_freqs.most_common(1)[0][0]
            best_freq = pair_freqs[best_pair]
            
            if best_freq == 0:
                break
            
            new_token = best_pair[0] + best_pair[1]
            new_id = len(self.vocab)
            self.vocab[new_token] = new_id
            self.inverse_vocab[new_id] = new_token
            self.merges.append(best_pair)
            
            pbar.set_description(f"Freq={best_freq}, {best_pair[0]!r}+{best_pair[1]!r}")
            
            affected_words = list(pair_to_words[best_pair])
            del pair_to_words[best_pair]
            del pair_freqs[best_pair]
            
            for old_word in affected_words:
                if old_word not in word_freqs:
                    continue
                    
                freq = word_freqs[old_word]
                
                if old_word in word_to_pairs:
                    for pos, pair in word_to_pairs[old_word]:
                        if pair in pair_freqs:
                            pair_freqs[pair] -= freq
                            if pair_freqs[pair] <= 0:
                                del pair_freqs[pair]
                        if pair in pair_to_words:
                            pair_to_words[pair].discard(old_word)
                
                new_word = []
                i = 0
                word_list = list(old_word)
                while i < len(word_list):
                    if i < len(word_list) - 1 and word_list[i] == best_pair[0] and word_list[i + 1] == best_pair[1]:
                        new_word.append(new_token)
                        i += 2
                    else:
                        new_word.append(word_list[i])
                        i += 1
                
                new_word = tuple(new_word)
                
                del word_freqs[old_word]
                if old_word in word_to_pairs:
                    del word_to_pairs[old_word]
                
                word_freqs[new_word] += freq
                
                if len(new_word) >= 2:
                    pairs_in_word = []
                    for i in range(len(new_word) - 1):
                        pair = (new_word[i], new_word[i + 1])
                        pair_freqs[pair] += freq
                        pair_to_words[pair].add(new_word)
                        pairs_in_word.append((i, pair))
                    word_to_pairs[new_word] = pairs_in_word
        
        # Индекс для быстрого encode
        self._build_merge_ranks()
        self.cache.clear()
        
        if verbose:
            print(f"Финальный размер словаря: {len(self.vocab)}")
    
    def _encode_word(self, word: str) -> List[int]:
        """Кодирование одного слова с кэшированием."""
        if word in self.cache:
            return self.cache[word]
        
        tokens = list(self._text_to_bytes(word))
        
        if len(tokens) <= 1:
            result = [self.vocab.get(t, self.vocab["<UNK>"]) for t in tokens]
            self.cache[word] = result
            return result
        
        # BPE через приоритетную очередь
        while len(tokens) > 1:
            min_rank = float('inf')
            min_idx = -1
            
            for i in range(len(tokens) - 1):
                pair = (tokens[i], tokens[i + 1])
                if pair in self.merge_ranks:
                    rank = self.merge_ranks[pair]
                    if rank < min_rank:
                        min_rank = rank
                        min_idx = i
            
            if min_idx == -1:
                break
            
            tokens = tokens[:min_idx] + [tokens[min_idx] + tokens[min_idx + 1]] + tokens[min_idx + 2:]
        
        result = [self.vocab.get(t, self.vocab["<UNK>"]) for t in tokens]
        
        if len(self.cache) < 100000:
            self.cache[word] = result
        
        return result
    
    def encode(self, text: str, add_special_tokens: bool = False) -> List[int]:
        tokens = []
        
        if add_special_tokens:
            tokens.append(self.vocab["<BOS>"])
        
        pretokens = self._pretokenize(text)
        
        for pretoken in pretokens:
            tokens.extend(self._encode_word(pretoken))
        
        if add_special_tokens:
            tokens.append(self.vocab["<EOS>"])
            
        return tokens
    
    def encode_batch(self, texts: List[str], add_special_tokens: bool = False, 
                     num_workers: int = 4) -> List[List[int]]:
        from concurrent.futures import ThreadPoolExecutor
        
        def encode_single(text):
            return self.encode(text, add_special_tokens)
        
        with ThreadPoolExecutor(max_workers=num_workers) as executor:
            results = list(executor.map(encode_single, texts))
        
        return results
    
    def decode(self, token_ids: List[int]) -> str:
        byte_tokens = []
        for token_id in token_ids:
            if token_id in self.inverse_vocab:
                token = self.inverse_vocab[token_id]
                if token not in self.special_tokens:
                    byte_tokens.append(token)
        
        byte_string = ''.join(byte_tokens).encode('latin-1')
        return byte_string.decode('utf-8', errors='replace')
    
    def get_vocab_size(self) -> int:
        return len(self.vocab)
    
    @property
    def pad_token_id(self) -> int:
        return self.vocab["<PAD>"]
    
    @property
    def bos_token_id(self) -> int:
        return self.vocab["<BOS>"]
    
    @property
    def eos_token_id(self) -> int:
        return self.vocab["<EOS>"]
    
    def save(self, path: str):
        data = {
            'vocab': self.vocab,
            'merges': self.merges,
            'special_tokens': self.special_tokens
        }
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
    
    @classmethod
    def load(cls, path: str) -> 'BPETokenizer':
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        tokenizer = cls(vocab_size=len(data['vocab']), special_tokens=data['special_tokens'])
        tokenizer.vocab = data['vocab']
        tokenizer.inverse_vocab = {int(v): k for k, v in data['vocab'].items()}
        tokenizer.merges = [tuple(m) for m in data['merges']]
        tokenizer._build_merge_ranks()
        return tokenizer

## 2. Подготовка обучающих данных

In [4]:
from datasets import load_dataset

print("Загрузка русской Википедии...")

dataset = load_dataset(
    "wikimedia/wikipedia",
    "20231101.ru",
    split="train",
    streaming=True, 
    trust_remote_code=True
)

NUM_ARTICLES = 50000
training_corpus = []

for i, item in enumerate(dataset):
    if i >= NUM_ARTICLES:
        break
    text = item['text']
    if len(text) > 200:
        training_corpus.append(text)
    if i % 10000 == 0:
        print(f"Загружено {i} статей...")

print(f"Загружено: {len(training_corpus)} статей")

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'wikimedia/wikipedia' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Загрузка русской Википедии...


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Загружено 0 статей...
Загружено 10000 статей...
Загружено 20000 статей...
Загружено 30000 статей...
Загружено 40000 статей...
Загружено: 49463 статей


In [5]:
random.shuffle(training_corpus)
test_texts = training_corpus[:5000]   
train_texts = training_corpus[5000:]    

print(f"Train: {len(train_texts)}")
print(f"Test: {len(test_texts)}")

Train: 44463
Test: 5000


In [6]:
VOCAB_SIZE = 5000

tokenizer = BPETokenizer(vocab_size=VOCAB_SIZE)
tokenizer.train(train_texts[:2000], verbose=True) 

print(f"\nРазмер словаря: {tokenizer.get_vocab_size()}")

Претокенизация и подсчет частот...


100%|██████████| 2000/2000 [00:12<00:00, 164.70it/s]


Уникальных претокенов: 289663
Построение индекса пар...
Выполняем 4739 слияний...


Freq=222, ' Ñ\x80ÐµÑ\x88'+'ÐµÐ½Ð¸Ñ\x8f': 100%|██████████| 4739/4739 [03:07<00:00, 25.24it/s]                         

Финальный размер словаря: 5000

Размер словаря: 5000


In [7]:
# Проверка токенизатора
test_text = "Привет, мир! Это тестовый текст."
encoded = tokenizer.encode(test_text)
decoded = tokenizer.decode(encoded)

print(f"Исходный: {test_text}")
print(f"Токены: {encoded}")
print(f"Декодированный: {decoded}")
print(f"Совпадение: {test_text == decoded}")

Исходный: Привет, мир! Это тестовый текст.
Токены: [2032, 840, 49, 2441, 38, 1952, 323, 454, 3346, 2397, 51]
Декодированный: Привет, мир! Это тестовый текст.
Совпадение: True


## 3. N-gram языковая модель

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")


class TextDataset(Dataset):
    def __init__(self, token_ids: List[int], seq_length: int = 128):
        self.token_ids = token_ids
        self.seq_length = seq_length
    
    def __len__(self):
        return max(0, len(self.token_ids) - self.seq_length)
    
    def __getitem__(self, idx):
        x = self.token_ids[idx:idx + self.seq_length]
        y = self.token_ids[idx + 1:idx + self.seq_length + 1]
        return torch.tensor(x), torch.tensor(y)


class SimpleLM(nn.Module):    
    def __init__(self, vocab_size: int, embed_dim: int = 256, context_size: int = 8):
        super().__init__()
        self.context_size = context_size
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim * context_size, vocab_size)
    
    def forward(self, x):
        # x: [batch, seq_len]
        batch_size, seq_len = x.shape
        
        emb = self.embedding(x)  # [batch, seq_len, embed_dim]
        
        # Для каждой позиции берём context_size предыдущих токенов
        outputs = []
        for i in range(seq_len):
            start = max(0, i - self.context_size + 1)
            ctx = emb[:, start:i+1, :]  # [batch, ctx_len, embed_dim]
            
            if ctx.shape[1] < self.context_size:
                pad_size = self.context_size - ctx.shape[1]
                pad = torch.zeros(batch_size, pad_size, emb.shape[2], device=x.device)
                ctx = torch.cat([pad, ctx], dim=1)
            
            ctx_flat = ctx.view(batch_size, -1)  # [batch, context_size * embed_dim]
            logits = self.fc(ctx_flat)  # [batch, vocab_size]
            outputs.append(logits)
        
        return torch.stack(outputs, dim=1)  # [batch, seq_len, vocab_size]


def train_model(model, train_loader, epochs=5, lr=0.001):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for x, y in pbar:
            x, y = x.to(device), y.to(device)
            
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_loss = total_loss / len(train_loader)
        ppl = math.exp(avg_loss)
        print(f"Epoch {epoch+1}: Loss={avg_loss:.4f}, PPL={ppl:.2f}")
    
    return model


def calculate_perplexity(model, data_loader):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
            total_loss += loss.item() * y.numel()
            total_tokens += y.numel()
    
    return math.exp(total_loss / total_tokens)


def generate(model, tokenizer, prompt: str, max_length: int = 50, temperature: float = 0.8):
    model.eval()
    tokens = tokenizer.encode(prompt)
    
    with torch.no_grad():
        for _ in range(max_length):
            ctx = tokens[-model.context_size:]
            x = torch.tensor([ctx]).to(device)
            
            logits = model(x)[0, -1, :] / temperature
            probs = torch.softmax(logits, dim=-1)
            
            next_token = torch.multinomial(probs, 1).item()
            tokens.append(next_token)
            
            if next_token == tokenizer.eos_token_id:
                break
    
    return tokenizer.decode(tokens)

Device: cuda


## 4. Обучение модели

In [9]:
print("Токенизация корпуса...")
all_tokens = []
for text in tqdm(train_texts[:5000]): 
    tokens = tokenizer.encode(text, add_special_tokens=True)
    all_tokens.extend(tokens[:500]) 

print(f"Всего токенов: {len(all_tokens):,}")

Токенизация корпуса...


100%|██████████| 5000/5000 [01:01<00:00, 81.55it/s] 

Всего токенов: 2,233,931


In [10]:
SEQ_LENGTH = 32 
BATCH_SIZE = 512

dataset = TextDataset(all_tokens, seq_length=SEQ_LENGTH)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

print(f"Батчей в эпохе: {len(train_loader)}")

Батчей в эпохе: 3927


In [11]:
model = SimpleLM(
    vocab_size=tokenizer.get_vocab_size(),
    embed_dim=128,  # Было 256
    context_size=8
)

In [12]:
model = train_model(model, train_loader, epochs=3, lr=0.002)

Epoch 2/3: 100%|██████████| 3927/3927 [12:26<00:00,  5.26it/s, loss=2.9447]


Epoch 2: Loss=3.1030, PPL=22.26


Epoch 3/3: 100%|██████████| 3927/3927 [12:25<00:00,  5.27it/s, loss=2.8895]

Epoch 3: Loss=2.9459, PPL=19.03


## 5. Оценка модели (Perplexity)

In [13]:
train_ppl = calculate_perplexity(model, train_loader)
val_ppl = calculate_perplexity(model, val_loader)
print(f"Train PPL: {train_ppl:.2f}")
print(f"Val PPL: {val_ppl:.2f}")

Train PPL: 17.43
Val PPL: 18.33


## 6. Генерация текста

In [14]:
for prompt in ["Машинное обучение", "Россия — это", "Нейронные сети"]:
    print(f"\n{prompt} → {generate(model, tokenizer, prompt)}")


Машинное обучение → Машинное обучение в Оренбург Монахельских (пах-Испийской) Гунге (Ге союз Турклось до недели власть его преемирались по лускому.

Первому причасточелю в производстве науч

Россия — это → Россия — это ученика).

Распространение 
Согласно данной сказке близокорд всё более расслоны. Ноус серьёзная рёка. За участие в молодых месяцев в году развержаться арестовавись с приказал, которая

Нейронные сети → Нейронные сети (см. : буквовый ей. Юрьевна, проводил с ним, и убили превращения в деревню в первом столицейском районе Грана Берклиана, критер, помимик, и


## 7. Сводка

### Токенизатор (BPE)
| Параметр | Значение |
|----------|----------|
| Тип | Byte Pair Encoding (BPE) |
| Паттерн претокенизации | GPT-4 (cl100k_base) |
| Источник | [tiktoken](https://github.com/openai/tiktoken) |
| Специальные токены | `<PAD>`, `<UNK>`, `<BOS>`, `<EOS>`, `<MASK>` |

### Данные
| Параметр | Значение |
|----------|----------|
| Корпус | Русская Википедия |
| Датасет | `wikimedia/wikipedia, 20231101.ru` |
| Текстов | 5,000 статей |
| Токенов | 2,233,931 |

### Модель
| Параметр | Значение |
|----------|----------|
| Архитектура | SimpleLM (Embedding + Linear) |
| Embedding dim | 128 |
| Context size | 8 токенов |

### Обучение
| Параметр | Значение |
|----------|----------|
| Эпох | 3 |
| Batch size | 512 |
| Sequence length | 32 |
| Learning rate | 0.002 |
| Оптимизатор | Adam |

In [15]:
print(f"""
Размер словаря:     {tokenizer.get_vocab_size():,}
Параметров модели:  {sum(p.numel() for p in model.parameters()):,}
Устройство:         {device}

Train Perplexity:   {train_ppl:.2f}
Val Perplexity:     {val_ppl:.2f}
Ratio (Val/Train):  {val_ppl/train_ppl:.2f}x
""")


Размер словаря:     5,000
Параметров модели:  5,765,000
Устройство:         cuda

Train Perplexity:   17.43
Val Perplexity:     18.33
Ratio (Val/Train):  1.05x



+BPE токенизатор успешно обучен на русском тексте

+Perplexity низкий и стабильный (≈18)

+Нет переобучения (Val/Train ≈ 1.05x)

+Генерация на русском языке работает

-Качество текста ограничено простой архитектурой и контекстом 8 токенов